In [1]:
import os
os.chdir('/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo')

In [2]:
import pandas as pd
from dotenv import load_dotenv
pd.set_option('display.max_columns',500)

In [3]:
load_dotenv()

True

## OfQual

In [15]:
ofqual_unit_df = pd.read_csv("../agents/ofqual_unit_details.csv")

In [16]:
ofqual_unit_df.sample()

,ofqual_id,overview,unit_id,unit_title,unit_description,unit_learning_outcomes
26268,603/0969/6,The Highfield Level 5 Certificate in Principle...,R/615/4418,The Role of the Commissioner for Wellbeing,This unit aims to develop the learner’s knowle...,1. Understand the values that inform commissio...


In [17]:
# Get ORG_ID -> Org name mapping
ofqual_org_df  = pd.read_csv("../docs/nos-ofqual/ofqual_orgs.csv")
org_id_mapping = dict(ofqual_org_df.iloc[:,:2].values)

In [18]:
# Previously merged ofqual unit data
ofqual_details_df = pd.read_csv("../docs/nos-ofqual/ofqual_details_20250127.csv.zip")
# Filter only available ofquals
ofqual_details_df =  ofqual_details_df[ofqual_details_df.status=='Available to learners']
keepcols = [
 'ofqual_id',
 'qualification_type',
 'qualification_level',
 'assessment_methods',
 'sector_subject_area',
 'awarding_organisation',
 'total_credits',
 'guided_learning_hours',
 'total_qualification_time',
]
ofqual_details_df = ofqual_details_df[keepcols]

# Get awarding organization
ofqual_details_df['awarding_organization'] = ofqual_details_df['awarding_organisation'].str.replace("/organisations/",'').str.strip().replace(org_id_mapping)

In [19]:
print(f"Units Extracted from PDFs: {len(ofqual_unit_df)}")
ofqual_unit_df = ofqual_unit_df[ofqual_unit_df.ofqual_id.isin(ofqual_details_df.ofqual_id.unique())]
print(f"Units after ofqual Matches: {len(ofqual_unit_df)}")

Units Extracted from PDFs: 30611
Units after ofqual Matches: 23909


In [20]:
ofqual_df = pd.merge(ofqual_unit_df,ofqual_details_df,on=['ofqual_id'],how='left')

In [21]:
ofqual_df.to_csv("../docs/rgcn/ofqual_units.csv",index=False)